## How much space do NYCHA residents have in their apartments? 
In this assignment, I explored data from the NYC Housing Authority (NYCHA). I worked on extracting, ingesting, and transforming the data. After that, I analyzed the data using summary statistics and visualization. Finally, I built a series of constant models and considered ways to select the best model from a finite set of candidates.

## Download data 
Go to https://data.cityofnewyork.us/Housing-Development/NYCHA-Development-Data-Book/evjd-dqpz/about_data and "Export" your data as "CSV". Save/move the `NYCHA_Development_Data_Book_20240903.csv` file into a convenient location (e.g. same as this notebook).

In [13]:
# libraries for data manipulation
import numpy as np
import pandas as pd

# libraries for data visualization
from matplotlib import pyplot as plt
import seaborn as sns

# jupyter extension to render charts inline
%matplotlib inline

## Section 1: Data loading and exploration


In [14]:
# read in the data in the CSV as-is into the variable raw_df.
raw_df = pd.read_csv("NYCHA_Development_Data_Book_20250127.csv")


In [15]:
# display the columns in raw_df
raw_df.columns

Index(['DATA AS OF', 'DEVELOPMENT', 'HUD AMP#', 'TDS#', 'CONSOLIDATED TDS#',
       'DEVELOPMENT EDP#', 'OPERATING EDP#', 'HUD #', 'PROGRAM', 'METHOD',
       'TYPE', 'NUMBER OF SECTION 8 TRANSITION APARTMENTS',
       'NUMBER OF CURRENT APARTMENTS', 'TOTAL NUMBER OF APARTMENTS',
       'NUMBER OF RENTAL ROOMS', 'AVG NO R/R PER APARTMENT',
       'POPULATION SECTION 8 TRANSITION', 'POPULATION PUBLIC HOUSING',
       'TOTAL POPULATION', 'TOTAL # OF FIXED INCOME HOUSEHOLD',
       'PERCENT FIXED INCOME HOUSEHOLDS', 'NUMBER OF RESIDENTIAL BLDGS',
       'NUMBER OF NON-RESIDENTIAL BLDGS', 'NUMBER OF STAIRHALLS',
       'NUMBER OF STORIES', 'TOTAL AREA SQ FT', 'ACRES', 'NET DEV AREA SQ FT',
       'EXCLUDING PARK ACRES', 'BLDG COVERAGE SQ FT', 'CUBAGE CU FT',
       'BLDG COVERAGE %', 'DENSITY', 'DEVELOPMENT COST', 'PER RENTAL ROOM',
       'AVG MONTHLY GROSS RENT', 'LOCATION STREET A', 'LOCATION STREET B',
       'LOCATION STREET C', 'LOCATION STREET D', 'BOROUGH',
       'COMMUNITY DISTIR

In [16]:
# show dataframe head
raw_df.head(5)


,DATA AS OF,DEVELOPMENT,HUD AMP#,TDS#,CONSOLIDATED TDS#,DEVELOPMENT EDP#,OPERATING EDP#,HUD #,PROGRAM,METHOD,...,US CONGRESSIONAL DISTRICT,NY STATE SENATE DISTRICT,NY STATE ASSEMBLY DISTRICT,NY CITY COUNCIL DISTRICT,COMPLETION DATE,FEDERALIZED DEVELOPMENT,SENIOR DEVELOPMENT,ELECTRICITY PAID BY RESIDENTS,PRIVATE MANAGEMENT,RAD TRANSFERRED DATE
0,1/1/2024,1010 EAST 178TH STREET,NY005011330,180,180,289,289,NY005090,FEDERAL,CONVENTIONAL,...,15,32,87,15,3/31/1971,NaN,NaN,NaN,NaN,NaN
1,1/1/2024,1162-1176 WASHINGTON AVENUE,NY005013080,233,308,354,344,NY005138,FEDERAL,TURNKEY,...,15,32,79,16,12/31/1975,NaN,NaN,NaN,NaN,NaN
2,1/1/2024,131 SAINT NICHOLAS AVENUE,NY005010970,154,97,264,261,NY005065,FEDERAL,CONVENTIONAL,...,13,30,70,9,3/31/1965,NaN,NaN,NaN,NaN,NaN
3,1/1/2024,1471 WATSON AVENUE,NY005010670,214,67,332,222,NY005162,FEDERAL,TURNKEY,...,14,32,85,17,12/31/1970,NaN,NaN,NaN,NaN,NaN
4,1/1/2024,154 WEST 84TH STREET,NY005013590,359,359,840,840,NY005270,FEDERAL,TURNKEY,...,12,47,69,6,3/31/1996,NaN,NaN,YES,YES,NaN


In [17]:
# preview subset of columns
raw_df[['DEVELOPMENT', 'TOTAL POPULATION', 'NUMBER OF CURRENT APARTMENTS', 'AVG MONTHLY GROSS RENT', 'NET DEV AREA SQ FT']] 

,DEVELOPMENT,TOTAL POPULATION,NUMBER OF CURRENT APARTMENTS,AVG MONTHLY GROSS RENT,NET DEV AREA SQ FT
0,1010 EAST 178TH STREET,413,205,$488,"88,172"
1,1162-1176 WASHINGTON AVENUE,141,65,$502,"18,987"
2,131 SAINT NICHOLAS AVENUE,157,88,$514,"29,359"
3,1471 WATSON AVENUE,116,96,$517,"39,937"
4,154 WEST 84TH STREET,65,35,$698,"9,621"
...,...,...,...,...,...
341,WASHINGTON HEIGHTS REHAB PHASE IV (D),60,32,$578,"8,743"
342,WEEKSVILLE GARDENS,697,249,$619,"141,365"
343,WILLIAMS PLAZA,"1,290",571,$496,"242,859"
344,WILLIAMSBURG,"2,873","1,564",$508,"927,103"


In [18]:
# define new columns 
raw_df["development"] = raw_df["DEVELOPMENT"]
raw_df["boro"] = raw_df["BOROUGH"]
raw_df["total_pop"] = raw_df["TOTAL POPULATION"]
raw_df["num_apts"] = raw_df["TOTAL NUMBER OF APARTMENTS"]
raw_df["avg_monthly_rent"] = raw_df["AVG MONTHLY GROSS RENT"]
raw_df["net_sqft"] = raw_df["NET DEV AREA SQ FT"]

# typecast to float
for col in raw_df[["total_pop","num_apts","avg_monthly_rent","net_sqft"]]:
    raw_df[col] = raw_df[col].str.replace('$', '').str.replace(',', '').astype(float)
    
raw_df=pd.DataFrame(raw_df).fillna(0)

In [19]:
# compute new derived columns
raw_df["ppl_per_apt"] = raw_df["total_pop"]/raw_df["num_apts"]
raw_df["sqft_per_apt"] = raw_df["net_sqft"]/raw_df["num_apts"]
raw_df["sqft_per_person"] = raw_df["net_sqft"]/raw_df["total_pop"]
raw_df["rent_per_sqft"] = raw_df["avg_monthly_rent"]/raw_df["net_sqft"]

In [20]:
# preview all the new columns
raw_df[[
    "development","boro","total_pop","num_apts",
    "avg_monthly_rent","net_sqft", 
    "sqft_per_apt","sqft_per_person","rent_per_sqft"
]]

,development,boro,total_pop,num_apts,avg_monthly_rent,net_sqft,sqft_per_apt,sqft_per_person,rent_per_sqft
0,1010 EAST 178TH STREET,BRONX,413.0,220.0,488.0,88172.0,400.781818,213.491525,0.005535
1,1162-1176 WASHINGTON AVENUE,BRONX,141.0,66.0,502.0,18987.0,287.681818,134.659574,0.026439
2,131 SAINT NICHOLAS AVENUE,MANHATTAN,157.0,100.0,514.0,29359.0,293.590000,187.000000,0.017507
3,1471 WATSON AVENUE,BRONX,116.0,96.0,517.0,39937.0,416.010417,344.284483,0.012945
4,154 WEST 84TH STREET,MANHATTAN,65.0,35.0,698.0,9621.0,274.885714,148.015385,0.072550
...,...,...,...,...,...,...,...,...,...
341,WASHINGTON HEIGHTS REHAB PHASE IV (D),MANHATTAN,60.0,32.0,578.0,8743.0,273.218750,145.716667,0.066110
342,WEEKSVILLE GARDENS,BROOKLYN,697.0,257.0,619.0,141365.0,550.058366,202.819225,0.004379
343,WILLIAMS PLAZA,BROOKLYN,1290.0,577.0,496.0,242859.0,420.899480,188.262791,0.002042
344,WILLIAMSBURG,BROOKLYN,2873.0,1621.0,508.0,927103.0,571.932758,322.695092,0.000548


In [21]:
#1.8
# define function to load and transform the NYCHA data
import pandas as pd
def read_and_transform_nycha_data(filepath: str) -> pd.DataFrame:
    """
    Reads the csv file and returns a clean table for preview

    This function takes the filepath from the user to read a csv file and cleans and organizes the data with necessary columns

    :param filepath: Filepath of the csv file.
    :type net_sqft: str
    :return: Dataframe with necessary data.
    :rtype: pd.Dataframe
    :raises ValueError: If `num_apts` or "total_pop" is zero or a negative number.
    """
   
    # reads the CSV file
    raw_df = pd.read_csv(filepath)
    
    # define new columns 
    raw_df["development"] = raw_df["DEVELOPMENT"]
    raw_df["boro"] = raw_df["BOROUGH"]
    raw_df["total_pop"] = raw_df["TOTAL POPULATION"]
    raw_df["num_apts"] = raw_df["TOTAL NUMBER OF APARTMENTS"]
    raw_df["avg_monthly_rent"] = raw_df["AVG MONTHLY GROSS RENT"]
    raw_df["net_sqft"] = raw_df["NET DEV AREA SQ FT"]
    
    # typecast to float
    for col in raw_df[["total_pop","num_apts","avg_monthly_rent","net_sqft"]]:
        raw_df[col] = raw_df[col].str.replace('$', '').str.replace(',', '').astype(float)

    #1.7 select newly defined columns and preview the dataframe
    raw_df["ppl_per_apt"] = raw_df["total_pop"]/raw_df["num_apts"]
    raw_df["sqft_per_apt"] = raw_df["net_sqft"]/raw_df["num_apts"]
    raw_df["sqft_per_person"] = raw_df["net_sqft"]/raw_df["total_pop"]
    raw_df["rent_per_sqft"] = raw_df["avg_monthly_rent"]/raw_df["net_sqft"]
    
    return raw_df[[
        "development","boro", "total_pop", "num_apts",
        "avg_monthly_rent", "net_sqft", 
        "ppl_per_apt", "sqft_per_apt", "sqft_per_person", "rent_per_sqft", 
    ]]

In [22]:
#load and transform NYCHA data into nycha_df
nycha_df = read_and_transform_nycha_data("NYCHA_Development_Data_Book_20250127.csv")
nycha_df

,development,boro,total_pop,num_apts,avg_monthly_rent,net_sqft,ppl_per_apt,sqft_per_apt,sqft_per_person,rent_per_sqft
0,1010 EAST 178TH STREET,BRONX,413.0,220.0,488.0,88172.0,1.877273,400.781818,213.491525,0.005535
1,1162-1176 WASHINGTON AVENUE,BRONX,141.0,66.0,502.0,18987.0,2.136364,287.681818,134.659574,0.026439
2,131 SAINT NICHOLAS AVENUE,MANHATTAN,157.0,100.0,514.0,29359.0,1.570000,293.590000,187.000000,0.017507
3,1471 WATSON AVENUE,BRONX,116.0,96.0,517.0,39937.0,1.208333,416.010417,344.284483,0.012945
4,154 WEST 84TH STREET,MANHATTAN,65.0,35.0,698.0,9621.0,1.857143,274.885714,148.015385,0.072550
...,...,...,...,...,...,...,...,...,...,...
341,WASHINGTON HEIGHTS REHAB PHASE IV (D),MANHATTAN,60.0,32.0,578.0,8743.0,1.875000,273.218750,145.716667,0.066110
342,WEEKSVILLE GARDENS,BROOKLYN,697.0,257.0,619.0,141365.0,2.712062,550.058366,202.819225,0.004379
343,WILLIAMS PLAZA,BROOKLYN,1290.0,577.0,496.0,242859.0,2.235702,420.899480,188.262791,0.002042
344,WILLIAMSBURG,BROOKLYN,2873.0,1621.0,508.0,927103.0,1.772363,571.932758,322.695092,0.000548


## Section 2: Exploratory data analysis
### Summary statistics


In [23]:
# display summary
nycha_df.describe()

,total_pop,num_apts,avg_monthly_rent,net_sqft,ppl_per_apt,sqft_per_apt,sqft_per_person,rent_per_sqft
count,338.000000,344.000000,338.000000,3.440000e+02,338.000000,344.000000,338.000000,338.000000
mean,1080.926036,532.409884,574.991124,3.025852e+05,1.939453,661.167243,398.317452,0.011712
std,1146.554629,558.107457,121.271442,3.813772e+05,0.497621,922.070514,780.085493,0.026941
min,3.000000,5.000000,229.000000,3.098000e+03,0.300000,83.686667,56.568627,0.000282
25%,216.500000,125.000000,530.000000,4.521125e+04,1.776529,323.148416,171.522437,0.001234
50%,549.000000,271.000000,583.000000,1.344110e+05,2.011832,434.983838,224.205534,0.003854
75%,1704.750000,805.250000,624.750000,4.639400e+05,2.196789,677.204463,366.246368,0.012456
max,4577.000000,2545.000000,1180.000000,2.141741e+06,3.651515,8227.600000,9281.200000,0.342479


### Data viz


In [24]:
# make pairplots
import seaborn as sns
sns.pairplot(nycha_df)

In [25]:
# compute correlation coefficients of the numeric columns
pearson_corr_nycha = nycha_df.corr(numeric_only=True)
pearson_corr_nycha

,total_pop,num_apts,avg_monthly_rent,net_sqft,ppl_per_apt,sqft_per_apt,sqft_per_person,rent_per_sqft
total_pop,1.000000,0.990073,0.098963,0.874212,0.206396,-0.091193,-0.138336,-0.324744
num_apts,0.990073,1.000000,0.058996,0.873595,0.121033,-0.096326,-0.135631,-0.328516
avg_monthly_rent,0.098963,0.058996,1.000000,0.084111,0.373707,0.424635,0.300986,0.224180
net_sqft,0.874212,0.873595,0.084111,1.000000,0.141920,0.050027,-0.030053,-0.293691
ppl_per_apt,0.206396,0.121033,0.373707,0.141920,1.000000,-0.081832,-0.275808,-0.084648
sqft_per_apt,-0.091193,-0.096326,0.424635,0.050027,-0.081832,1.000000,0.903775,-0.051150
sqft_per_person,-0.138336,-0.135631,0.300986,-0.030053,-0.275808,0.903775,1.000000,0.004254
rent_per_sqft,-0.324744,-0.328516,0.224180,-0.293691,-0.084648,-0.051150,0.004254,1.000000


## Square feet per apartment -- constant models


In [26]:
# plot histogram of sqft_per_person
import seaborn as sns
sns.displot(nycha_df, x='sqft_per_person', aspect=3)

### Constant models


In [27]:
# create constant models
cm_mean = nycha_df['sqft_per_person'].mean()
cm_median = nycha_df['sqft_per_person'].median()

# print predictions
print(f"  mean sqft per person: {cm_mean}")
print(f"median sqft per person: {cm_median}")

  mean sqft per person: 398.3174515872763
median sqft per person: 224.20553359683794


### Model validation

In [28]:
# define loss functions
import numpy as np
def mae_loss(y_pred, y_true):
    """
    Calculates the Mean Absolute Error

    This function computes the average of the absolute differences between 
    predicted values (`y_pred`) and actual values (`y_true`)

    :param y_pred: Series of predicted values
    :type y_pred: pandas.Series
    :param y_true: Series of true values
    :type y_true: pandas.Series
    :return: Mean Absolute Error
    :rtype: float
    """
    mae = (y_true - y_pred).abs().mean()
    return mae

def rmse_loss(y_pred, y_true):
    """
    Calculates the Root Mean Squared Error 

    This function computes the square root of the average squared differences between 
    predicted values (`y_pred`) and actual values (`y_true`)

    :param y_pred:Series of predicted values
    :type y_pred: pandas.Series
    :param y_true: Series of true values
    :type y_true: pandas.Series
    :return: Root Mean Squared Error
    :rtype: float
    """
    rmse = np.sqrt(((y_true - y_pred) ** 2).mean())
    return rmse

In [29]:
# compute both loss functions for each model
y_true = nycha_df['sqft_per_person']
cm_mean = y_true.mean()
cm_median = y_true.median()

cm_mean_mae = mae_loss(cm_mean, y_true)
cm_mean_rmse = rmse_loss(cm_mean, y_true)

cm_median_mae = mae_loss(cm_median, y_true)
cm_median_rmse = rmse_loss(cm_median, y_true)

In [30]:
# print losses
print("  mean constant model MAE:", cm_mean_mae)
print("  mean constant model RMSE:", cm_mean_rmse)

print(f"median constant model MAE: {cm_median_mae}")
print(f"median constant model RMSE: {cm_median_rmse}")

  mean constant model MAE: 292.8912232585764
  mean constant model RMSE: 778.9306654274562
median constant model MAE: 233.52137433708887
median constant model RMSE: 798.1528309350089


### Minimize your loss functions


In [31]:
# function to compute losses over many constant models

def compute_losses(y_true: pd.Series):
    """Compute MAE and RMSE losses over a range of theta values.
    
    :y_true: pandas.Series with true data.
    :returns: pandas.DataFrame with MAE and RMSE values for each theta.
    """
    mae_list = []
    rmse_list = []
    thetas = np.linspace(0, 1000, 1001)
    for i in thetas:
        mae = (y_true - i).abs().mean()
        rmse = np.sqrt(((y_true - i) ** 2).mean())
        
        # append the results to the lists
        mae_list.append(mae)
        rmse_list.append(rmse)

        # Create a DataFrame to store theta, MAE, and RMSE
    results_df = pd.DataFrame({
        "theta": thetas,
        "mae": mae_list,
        "rmse": rmse_list
    })
        
    return results_df
   
    

In [32]:
# compute losses over many constant models
losses_df = compute_losses(nycha_df['sqft_per_person'])
losses_df.head(5)

,theta,mae,rmse
0,0.0,398.317452,874.865575
1,1.0,397.317452,874.410738
2,2.0,396.317452,873.956809
3,3.0,395.317452,873.503789
4,4.0,394.317452,873.051679


In [33]:
# plot losses at each theta

sns.lineplot(
    # displays the MAE lineplot
    data=losses_df,x="theta",y='mae', label="MAE"
)

sns.lineplot(
    # displays the RMSE lineplot 
    data = losses_df,x="theta",y='rmse',label="RMSE"
)

plt.ylabel("loss")

Text(29.000000000000007, 0.5, 'loss')

In [34]:
# select row that minimizes MAE
losses_df.loc[losses_df['mae'].idxmin()]

theta    224.000000
mae      233.521374
rmse     798.197692
Name: 224, dtype: float64

In [35]:
# select row that minimizes RMSE
losses_df.loc[losses_df['rmse'].idxmin()]

theta    398.000000
mae      292.705789
rmse     778.930730
Name: 398, dtype: float64